<a href="https://colab.research.google.com/github/prasanth5reddy/D2L/blob/master/Computer%20Vision/transposed_convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
w_dir = '/content/drive/My Drive/Colab/D2L.AI/'
sys.path.append(w_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing Libraries

In [2]:
!pip uninstall mxnet && pip install mxnet-cu100

Importing Libraries

In [0]:
from mxnet import nd, init
from mxnet.gluon import nn
import d2l

Basic 2D transposed convolution

In [0]:
def trans_conv(X, K):
  h, w = K.shape
  Y = nd.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      Y[i: i + h, j: j + w] += X[i, j] * K
  return Y

In [5]:
X = nd.array([[0,1], [2,3]])
K = nd.array([[0,1], [2,3]])
trans_conv(X, K)


[[ 0.  0.  1.]
 [ 0.  4.  6.]
 [ 4. 12.  9.]]
<NDArray 3x3 @cpu(0)>

In [6]:
X, K = X.reshape((1, 1, 2, 2)),  K.reshape((1, 1, 2, 2))
tconv = nn.Conv2DTranspose(1, kernel_size=2)
tconv.initialize(init.Constant(K))
tconv(X)


[[[[ 0.  0.  1.]
   [ 0.  4.  6.]
   [ 4. 12.  9.]]]]
<NDArray 1x1x3x3 @cpu(0)>

Padding, Strides and Channels

In [7]:
tconv = nn.Conv2DTranspose(1, kernel_size=2, padding=1)
tconv.initialize(init.Constant(K))
tconv(X)


[[[[4.]]]]
<NDArray 1x1x1x1 @cpu(0)>

In [8]:
tconv = nn.Conv2DTranspose(1, kernel_size=2, strides=2)
tconv.initialize(init.Constant(K))
tconv(X)


[[[[0. 0. 0. 1.]
   [0. 0. 2. 3.]
   [0. 2. 0. 3.]
   [4. 6. 6. 9.]]]]
<NDArray 1x1x4x4 @cpu(0)>

In [9]:
X = nd.random.uniform(shape=(1, 10, 16, 16))
conv = nn.Conv2D(30, kernel_size=5, padding=2, strides=3)
tconv = nn.Conv2DTranspose(10, kernel_size=5, padding=2, strides=3)
conv.initialize()
tconv.initialize()
tconv(conv(X)).shape == X.shape

True

Analogy to matrix transposition

In [10]:
X = nd.arange(9).reshape((3,3))
K = nd.array([[0,1], [2,3]])
Y = d2l.corr2d(X, K)
Y


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

In [11]:
def kernel2matrix(K):
    k, W = nd.zeros(5), nd.zeros((4, 9))
    k[:2], k[3:5] = K[0,:], K[1,:]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W


[[0. 1. 0. 2. 3. 0. 0. 0. 0.]
 [0. 0. 1. 0. 2. 3. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 2. 3. 0.]
 [0. 0. 0. 0. 0. 1. 0. 2. 3.]]
<NDArray 4x9 @cpu(0)>

In [12]:
Y == nd.dot(W, X.reshape((-1))).reshape((2,2))


[[1. 1.]
 [1. 1.]]
<NDArray 2x2 @cpu(0)>

In [13]:
X = nd.array([[0,1], [2,3]])
Y = trans_conv(X, K)
Y == nd.dot(W.T, X.reshape((-1))).reshape((3,3))


[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
<NDArray 3x3 @cpu(0)>